In [20]:
#import necessary libraries.
import cv2
import numpy as np
import imutils
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C://Program Files (x86)//Tesseract-OCR//tesseract.exe'

#read the image & resize it.
img1 = cv2.imread("C://Users//user//Desktop//image//carimg3.jpg",cv2.IMREAD_COLOR)
img1 = cv2.resize(img1,(620,480))

#convert the image into grayscale.
gray = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

#Apply bilateral filter and canny edge detection.
gray = cv2.bilateralFilter(gray,13,15,15)
edged = cv2.Canny(gray,30,200)

#find contours  use a copy of the image since findContours can alters the image. 
#cv2.CHAIN_APPROX_SIMPLE is used to store only the boundary points of the contours.
contours = cv2.findContours(edged.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) 
contours = imutils.grab_contours(contours)

#sort the contours from big to small.
contours = sorted(contours, key = cv2.contourArea,reverse = True)[:10]
screenCnt = None

#loop through the contours and find contours with four sides and closed figure.
for c in contours:
    peri = cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,0.018*peri,True)
    #if approximated contours have four points means then we have found our screen.
    if len(approx) == 4:
        screenCnt = approx
        break

#if there is no contours are detected then print the result.
if screenCnt is None:
    detected = 0
    print("no contours detected")
else:
    detected = 1

    #draw over the detected contour.
if detected == 1:
    image = cv2.drawContours(img1,[screenCnt],-1,(0,255,0),3)
    
#mask the rest of the contours since they are useless.
mask = np.zeros(gray.shape,np.uint8)
newimage = cv2.drawContours(mask,[screenCnt],0,255,-1)
newimage = cv2.bitwise_and(img1,img1,mask=mask)
cv2.imshow("maskedimg",newimage)

#cropping segment.
(x,y) = np.where(mask == 255)
(topx,topy) = (np.min(x),np.min(y))
(bottomx,bottomy) = (np.max(x),np.max(y))
cropped = gray[topx:bottomx+1 , topy:bottomy+1]

#read the number plate.
text = pytesseract.image_to_string(cropped,config='--psm11')
print("detected no. is:",text)

#resize the resulted image.
img1 =cv2.resize(img1,(500,300))
cropped = cv2.resize(cropped,(400,200))

#display the result.
cv2.imshow("car",img1)
cv2.imshow("cropped",cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()

detected no. is: 2376 S7


